# Lab | Handling Data Imbalance in Classification Models
For this lab and in the next lessons we will build a model on customer churn binary classification problem. You will be using files_for_lab/Customer-Churn.csv file.

**Scenario**
You are working as an analyst with this internet service provider. You are provided with this historical data about your company's customers and their churn trends. Your task is to build a machine learning model that will help the company identify customers that are more likely to default/churn and thus prevent losses from such customers.

**Instructions**
In this lab, we will first take a look at the degree of imbalance in the data and correct it using the techniques we learned on the class.

Here is the list of steps to be followed (building a simple model without balancing the data):

- Import the required libraries and modules that you would need.
- Read that data into Python and call the dataframe churnData.
- Check the datatypes of all the columns in the data. You would see that the column TotalCharges is object type. Convert this column into numeric type using pd.to_numeric function.
- Check for null values in the dataframe. Replace the null values.
- Use the following features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges:
    - Scale the features either by using normalizer or a standard scaler.
    - Split the data into a training set and a test set.
    - Fit a logistic regression model on the training data.
    - Check the accuracy on the test data.
*Note: So far we have not balanced the data.*

Managing imbalance in the dataset

- Check for the imbalance.
- Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
- Each time fit the model and see how the accuracy of the model is.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error as unkind_squared_error, r2_score
from sklearn import neighbors
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score

In [2]:
churnData = pd.read_csv('files_for_lab/Customer-Churn.csv')

In [3]:
churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [4]:
churnData.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [5]:
churnData.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [6]:
churnData[churnData['TotalCharges']==" "].head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
488,Female,0,Yes,Yes,0,No,Yes,No,Yes,Yes,Yes,No,Two year,52.55,,No
753,Male,0,No,Yes,0,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,20.25,,No
936,Female,0,Yes,Yes,0,Yes,Yes,Yes,Yes,No,Yes,Yes,Two year,80.85,,No
1082,Male,0,Yes,Yes,0,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,25.75,,No
1340,Female,0,Yes,Yes,0,No,Yes,Yes,Yes,Yes,Yes,No,Two year,56.05,,No


TotalCharges is " " when it's a new customer with 0 tenure. Lets set these to 0.

In [7]:
churnData['TotalCharges'] = list(map(lambda x:0 if x == " " else x, churnData['TotalCharges']))
churnData['TotalCharges'] = pd.to_numeric(churnData['TotalCharges'])

In [8]:
churnData.isna().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [9]:
churnData['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [10]:
# as we are doing logistic regression I feel the target should be numeric too... oh, i got my linear and logistic confused, but then did all the things anyways...
churnData['Churn'] = list(map(lambda x:0 if x == "No" else 1, churnData['Churn']))

In [11]:
churnData['Churn'].value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

In [12]:
# select features
X = churnData[['tenure','SeniorCitizen','MonthlyCharges','TotalCharges']]

In [13]:
# train test split - doing this first to prevent bleeding info into our training data from the test data
X_train, X_test, y_train, y_test = train_test_split(X, churnData['Churn'], test_size=0.2, random_state=79)

In [14]:
# scale - now doing scaling to be fair to be fair TOOO BEEEE FAAAIIRRRRR .... https://www.youtube.com/watch?v=jv7jcciKB_s
transformer = MinMaxScaler().fit(X_train)
X_train = transformer.transform(X_train)
X_test = transformer.transform(X_test)
# y_train = y_train.reset_index()
# y_test = y_test.reset_index()

In [15]:
X_added_constant = sm.add_constant(X_train)
model = sm.OLS(y_train,X_added_constant).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Churn   R-squared:                       0.236
Model:                            OLS   Adj. R-squared:                  0.235
Method:                 Least Squares   F-statistic:                     433.8
Date:                Wed, 01 Dec 2021   Prob (F-statistic):               0.00
Time:                        22:22:13   Log-Likelihood:                -2599.6
No. Observations:                5634   AIC:                             5209.
Df Residuals:                    5629   BIC:                             5242.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1898      0.015     12.720      0.000       0.161       0.219
x1            -0.1818      0.036     -5.038      0.000      -0.253      -0.111
x2             0.1002      0.014      7.032      0.000       0.072       0.128
x3             0.6861      0.031     22.046      0.000       0.625       0.747
x4            -0.6826      0.061    -11.271      0.000      -0.801      -0.564
==============================================================================
Omnibus:                      410.681   Durbin-Watson:                   2.046
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              480.118
Skew:                           0.697   Prob(JB):                    5.54e-105
Kurtosis:                       2.677   Cond. No.                         18.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
lg = linear_model.LogisticRegression(random_state=0)
model = lg.fit(X_train,y_train)
    
predictions_train = lg.predict(X_train)
predictions_test = lg.predict(X_test)
       
print("Accuracy (Train): \t" + str(model.score(X_train, y_train)))
print("Accuracy (Test): \t" + str(model.score(X_test, y_test)))
print("Precision (Test): \t" + str(precision_score(y_test, model.predict(X_test))))
print("Recall (Test): \t\t" + str(recall_score(y_test, model.predict(X_test))))

Accuracy (Train): 	0.7939297124600639
Accuracy (Test): 	0.7799858055358411
Precision (Test): 	0.6691729323308271
Recall (Test): 		0.445


In [17]:
confusion_matrix(y_test, predictions_test)

array([[921,  88],
       [222, 178]], dtype=int64)

In [18]:
def model_this(models, X_train, y_train, X_test, y_test, note, results):
    for m in models:
        model = m.fit(X_train,y_train)
        r_train = recall_score(y_train, model.predict(X_train))
        r_test = recall_score(y_test, model.predict(X_test))
        p_train = precision_score(y_train, model.predict(X_train))
        p_test = precision_score(y_test, model.predict(X_test))
        results.loc[len(results)] = [m, note, m.score(X_train, y_train), m.score(X_test, y_test), r_train, r_test, p_train, p_test, confusion_matrix(y_test, model.predict(X_test))]
    return results

In [19]:
# models with basically random parameters...
KNR = KNeighborsRegressor(n_neighbors=6)
lm = linear_model.LinearRegression()
MLP = MLPRegressor()
lr = LogisticRegression(random_state=0)
clf = neighbors.KNeighborsClassifier(n_neighbors=10, weights='uniform')

models = []
#models.extend([lm, KNR, MLP, lr, clf])
#models.extend([KNR, lr, clf])
models.extend([lr, clf])

In [20]:
X_train = pd.DataFrame(X_train, columns=['tenure','SeniorCitizen','MonthlyCharges','TotalCharges'])
X_test = pd.DataFrame(X_test, columns=['tenure','SeniorCitizen','MonthlyCharges','TotalCharges'])

In [21]:
results = pd.DataFrame(columns=['Model','Note','Train Accuracy','Test Accuracy', 'Train Recall', 'Test Recall', 'Train Precision', 'Test Precision','Confusion Matrix'])
results = model_this(models, X_train, y_train, X_test, y_test, "no-sampling",results)

In [22]:
results

,Model,Note,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Confusion Matrix
0,LogisticRegression(random_state=0),no-sampling,0.793930,0.779986,0.447924,0.4450,0.652778,0.669173,"[[921, 88], [222, 178]]"
1,KNeighborsClassifier(n_neighbors=10),no-sampling,0.818601,0.770759,0.466984,0.3825,0.741622,0.668122,"[[933, 76], [247, 153]]"


In [23]:
churnData['Churn'].value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

## SMOTE THINE IMBALANCE

In [24]:
smite = SMOTE(random_state=100,k_neighbors=5)
y_train = y_train.reset_index(drop=True)
to_be_smoted = pd.concat([pd.DataFrame(X_train), pd.DataFrame(y_train)], axis=1)

In [25]:
to_be_smoted.columns = ['tenure','SeniorCitizen','MonthlyCharges','TotalCharges','Churn']
to_be_smoted

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges,Churn
0,0.597222,0.0,0.065369,0.123221,0
1,0.027778,0.0,0.014471,0.004709,0
2,0.041667,0.0,0.165170,0.011641,0
3,0.472222,0.0,0.701597,0.345460,0
4,0.194444,0.0,0.509980,0.106093,0
...,...,...,...,...,...
5629,0.263889,0.0,0.016467,0.044008,0
5630,0.208333,0.0,0.847305,0.177298,0
5631,0.930556,0.0,0.708583,0.695301,0
5632,0.847222,0.0,0.016966,0.147246,0


In [26]:
X_train,y_train = smite.fit_resample(to_be_smoted,to_be_smoted['Churn'])

In [27]:
X_train = X_train.drop(['Churn'],axis=1)

In [30]:
results = model_this(models, X_train, y_train, X_test, y_test, "SMOTE",results)
results

,Model,Note,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Confusion Matrix
0,LogisticRegression(random_state=0),no-sampling,0.793930,0.779986,0.447924,0.4450,0.652778,0.669173,"[[921, 88], [222, 178]]"
1,KNeighborsClassifier(n_neighbors=10),no-sampling,0.818601,0.770759,0.466984,0.3825,0.741622,0.668122,"[[933, 76], [247, 153]]"
2,LogisticRegression(random_state=0),SMOTE,0.740816,0.712562,0.750300,0.6925,0.736334,0.495528,"[[727, 282], [123, 277]]"
3,KNeighborsClassifier(n_neighbors=10),SMOTE,0.817407,0.730305,0.836014,0.6450,0.806019,0.520161,"[[771, 238], [142, 258]]"


## Downsampling

In [31]:
le_data = to_be_smoted

category_0 = le_data[le_data['Churn'] == 0]
category_1 = le_data[le_data['Churn'] == 1]

In [32]:
category_0_undersampled = resample(category_0, 
                                   replace=False, 
                                   n_samples = len(category_1))
data_downsampled = pd.concat([category_0_undersampled, category_1], axis=0)
data_downsampled['Churn'].value_counts()

0    1469
1    1469
Name: Churn, dtype: int64

In [33]:
y = data_downsampled[['Churn']]
X = data_downsampled.drop(['Churn'], axis=1)
results = model_this(models, X, np.ravel(y), X_test, y_test, "Downsampling",results)
results

,Model,Note,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Confusion Matrix
0,LogisticRegression(random_state=0),no-sampling,0.793930,0.779986,0.447924,0.4450,0.652778,0.669173,"[[921, 88], [222, 178]]"
1,KNeighborsClassifier(n_neighbors=10),no-sampling,0.818601,0.770759,0.466984,0.3825,0.741622,0.668122,"[[933, 76], [247, 153]]"
2,LogisticRegression(random_state=0),SMOTE,0.740816,0.712562,0.750300,0.6925,0.736334,0.495528,"[[727, 282], [123, 277]]"
3,KNeighborsClassifier(n_neighbors=10),SMOTE,0.817407,0.730305,0.836014,0.6450,0.806019,0.520161,"[[771, 238], [142, 258]]"
4,LogisticRegression(random_state=0),Downsampling,0.736555,0.711143,0.746086,0.6925,0.732131,0.493761,"[[725, 284], [123, 277]]"
5,KNeighborsClassifier(n_neighbors=10),Downsampling,0.775357,0.720369,0.770592,0.6875,0.778007,0.505515,"[[740, 269], [125, 275]]"


## Upsampling

In [34]:
category_1_oversampled = resample(category_1, 
                                  replace=True, 
                                  n_samples = len(category_0))
data_upsampled = pd.concat([category_0, category_1_oversampled], axis=0)
data_upsampled['Churn'].value_counts()

0    4165
1    4165
Name: Churn, dtype: int64

In [35]:
y = data_upsampled[['Churn']]
X = data_upsampled.drop(['Churn'], axis=1)
results = model_this(models, X, np.ravel(y), X_test, y_test, "Upsampling",results)
results

,Model,Note,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Confusion Matrix
0,LogisticRegression(random_state=0),no-sampling,0.793930,0.779986,0.447924,0.4450,0.652778,0.669173,"[[921, 88], [222, 178]]"
1,KNeighborsClassifier(n_neighbors=10),no-sampling,0.818601,0.770759,0.466984,0.3825,0.741622,0.668122,"[[933, 76], [247, 153]]"
2,LogisticRegression(random_state=0),SMOTE,0.740816,0.712562,0.750300,0.6925,0.736334,0.495528,"[[727, 282], [123, 277]]"
3,KNeighborsClassifier(n_neighbors=10),SMOTE,0.817407,0.730305,0.836014,0.6450,0.806019,0.520161,"[[771, 238], [142, 258]]"
4,LogisticRegression(random_state=0),Downsampling,0.736555,0.711143,0.746086,0.6925,0.732131,0.493761,"[[725, 284], [123, 277]]"
5,KNeighborsClassifier(n_neighbors=10),Downsampling,0.775357,0.720369,0.770592,0.6875,0.778007,0.505515,"[[740, 269], [125, 275]]"
6,LogisticRegression(random_state=0),Upsampling,0.740216,0.712562,0.748619,0.6900,0.736246,0.495512,"[[728, 281], [124, 276]]"
7,KNeighborsClassifier(n_neighbors=10),Upsampling,0.807923,0.721789,0.833133,0.6550,0.793143,0.507752,"[[755, 254], [138, 262]]"


In [36]:
results[results['Test Accuracy'] == results['Test Accuracy'].max()]

,Model,Note,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Confusion Matrix
0,LogisticRegression(random_state=0),no-sampling,0.79393,0.779986,0.447924,0.445,0.652778,0.669173,"[[921, 88], [222, 178]]"


In [37]:
results.sort_values(by=['Test Accuracy'], ascending=False, axis=0)

,Model,Note,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Confusion Matrix
0,LogisticRegression(random_state=0),no-sampling,0.793930,0.779986,0.447924,0.4450,0.652778,0.669173,"[[921, 88], [222, 178]]"
1,KNeighborsClassifier(n_neighbors=10),no-sampling,0.818601,0.770759,0.466984,0.3825,0.741622,0.668122,"[[933, 76], [247, 153]]"
3,KNeighborsClassifier(n_neighbors=10),SMOTE,0.817407,0.730305,0.836014,0.6450,0.806019,0.520161,"[[771, 238], [142, 258]]"
7,KNeighborsClassifier(n_neighbors=10),Upsampling,0.807923,0.721789,0.833133,0.6550,0.793143,0.507752,"[[755, 254], [138, 262]]"
5,KNeighborsClassifier(n_neighbors=10),Downsampling,0.775357,0.720369,0.770592,0.6875,0.778007,0.505515,"[[740, 269], [125, 275]]"
2,LogisticRegression(random_state=0),SMOTE,0.740816,0.712562,0.750300,0.6925,0.736334,0.495528,"[[727, 282], [123, 277]]"
6,LogisticRegression(random_state=0),Upsampling,0.740216,0.712562,0.748619,0.6900,0.736246,0.495512,"[[728, 281], [124, 276]]"
4,LogisticRegression(random_state=0),Downsampling,0.736555,0.711143,0.746086,0.6925,0.732131,0.493761,"[[725, 284], [123, 277]]"


In [38]:
results.sort_values(by=['Test Recall'], ascending=False, axis=0)

,Model,Note,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Confusion Matrix
2,LogisticRegression(random_state=0),SMOTE,0.740816,0.712562,0.750300,0.6925,0.736334,0.495528,"[[727, 282], [123, 277]]"
4,LogisticRegression(random_state=0),Downsampling,0.736555,0.711143,0.746086,0.6925,0.732131,0.493761,"[[725, 284], [123, 277]]"
6,LogisticRegression(random_state=0),Upsampling,0.740216,0.712562,0.748619,0.6900,0.736246,0.495512,"[[728, 281], [124, 276]]"
5,KNeighborsClassifier(n_neighbors=10),Downsampling,0.775357,0.720369,0.770592,0.6875,0.778007,0.505515,"[[740, 269], [125, 275]]"
7,KNeighborsClassifier(n_neighbors=10),Upsampling,0.807923,0.721789,0.833133,0.6550,0.793143,0.507752,"[[755, 254], [138, 262]]"
3,KNeighborsClassifier(n_neighbors=10),SMOTE,0.817407,0.730305,0.836014,0.6450,0.806019,0.520161,"[[771, 238], [142, 258]]"
0,LogisticRegression(random_state=0),no-sampling,0.793930,0.779986,0.447924,0.4450,0.652778,0.669173,"[[921, 88], [222, 178]]"
1,KNeighborsClassifier(n_neighbors=10),no-sampling,0.818601,0.770759,0.466984,0.3825,0.741622,0.668122,"[[933, 76], [247, 153]]"


In [39]:
results.sort_values(by=['Test Precision'], ascending=False, axis=0)

,Model,Note,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Confusion Matrix
0,LogisticRegression(random_state=0),no-sampling,0.793930,0.779986,0.447924,0.4450,0.652778,0.669173,"[[921, 88], [222, 178]]"
1,KNeighborsClassifier(n_neighbors=10),no-sampling,0.818601,0.770759,0.466984,0.3825,0.741622,0.668122,"[[933, 76], [247, 153]]"
3,KNeighborsClassifier(n_neighbors=10),SMOTE,0.817407,0.730305,0.836014,0.6450,0.806019,0.520161,"[[771, 238], [142, 258]]"
7,KNeighborsClassifier(n_neighbors=10),Upsampling,0.807923,0.721789,0.833133,0.6550,0.793143,0.507752,"[[755, 254], [138, 262]]"
5,KNeighborsClassifier(n_neighbors=10),Downsampling,0.775357,0.720369,0.770592,0.6875,0.778007,0.505515,"[[740, 269], [125, 275]]"
2,LogisticRegression(random_state=0),SMOTE,0.740816,0.712562,0.750300,0.6925,0.736334,0.495528,"[[727, 282], [123, 277]]"
6,LogisticRegression(random_state=0),Upsampling,0.740216,0.712562,0.748619,0.6900,0.736246,0.495512,"[[728, 281], [124, 276]]"
4,LogisticRegression(random_state=0),Downsampling,0.736555,0.711143,0.746086,0.6925,0.732131,0.493761,"[[725, 284], [123, 277]]"


**Conclusion**

*Accuracy/Overall score drops a bit when sampling is applied, this is due to our precision taking a bit drop, however, recall increases. So it makes sense to do this if the focus is on reducing the number of false-negatives...but comes at a cost of an increase to false-positives.*